<a href="https://colab.research.google.com/github/vivi1231/0724/blob/master/news_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile
f = zipfile.ZipFile("drive/My Drive/0724/chinese_news_trans.zip")
f.extractall("train")
f = zipfile.ZipFile("drive/My Drive/0724/chinese_news_test.zip")
f.extractall("test")


In [ ]:
import glob
import os
import pandas as pd

def getdf(targetdn):
  datas = {
      "path":[],
      "label":[]    
  }
  base = ""
  p = os.path.join(base, targetdn, "*")
  for dn in glob.glob(p):
    name = os.path.split(dn)[-1]
    fn = os.path.join(dn, "*.[tT][xX][tT]")
    fns = glob.glob(fn)
    datas["label"] = datas["label"] + [name] * len(fns)
    datas["path"] = datas["path"] + fns
  return pd.DataFrame(datas)

In [ ]:
train_df = getdf("train/chinese_news_trans")
test_df = getdf("test/chinese_news_test")

test_df


,path,label
0,test/chinese_news_test/軍事/8240.txt,軍事
1,test/chinese_news_test/軍事/8249.txt,軍事
2,test/chinese_news_test/軍事/8245.txt,軍事
3,test/chinese_news_test/軍事/8244.txt,軍事
4,test/chinese_news_test/軍事/8247.txt,軍事
...,...,...
96,test/chinese_news_test/政治/9505.TXT,政治
97,test/chinese_news_test/政治/9501.TXT,政治
98,test/chinese_news_test/政治/9500.TXT,政治
99,test/chinese_news_test/政治/9497.TXT,政治


In [ ]:
def read_content(fn):
  with open(fn, "r", encoding="utf-8") as f:
    content = f.read()
  return content
train_df["content"] = train_df["path"].apply(read_content)
test_df["content"] = test_df["path"].apply(read_content)

In [ ]:
test_df

,path,label,content
0,test/chinese_news_test/軍事/8240.txt,軍事,美海軍成功演示飛機在空中控制無人機(附圖) \n\n--------------------...
1,test/chinese_news_test/軍事/8249.txt,軍事,\n軍事評論：“巴格達之戰”考驗美軍巷戰武器裝備(2) \n\n--------------...
2,test/chinese_news_test/軍事/8245.txt,軍事,軍事評論：美英聯軍地面作戰裝備特點分析(2) \n\n-------------------...
3,test/chinese_news_test/軍事/8244.txt,軍事,在伊拉克戰爭中使用的空襲精確制導彈藥(1) \n\n--------------------...
4,test/chinese_news_test/軍事/8247.txt,軍事,軍事評論：從巷戰或地道戰看單兵夜視裝備的發展 \n\n-------------------...
...,...,...,...
96,test/chinese_news_test/政治/9505.TXT,政治,楊主席提出促進中拉關係的５項建議\n新華社蒙得維的亞５月２３日電正在這...
97,test/chinese_news_test/政治/9501.TXT,政治,阿拉伯外長會議閉幕\n新華社巴格達５月２４日電為期２天的阿拉伯外長會\...
98,test/chinese_news_test/政治/9500.TXT,政治,外事簡訊\n新華社北京５月２４日電外事簡訊\n 全國人大常委會副...
99,test/chinese_news_test/政治/9497.TXT,政治,第３次安第斯集團首腦會議閉幕\n新華社庫斯科（祕魯）５月２３日電第３次...


In [ ]:
# 正確度、輸入新聞後判別類型

In [ ]:
u = train_df["label"].unique()
trans = {n:i for i, n in enumerate(u)}
trans_r = {i:n for i, n in enumerate(u)}
y_train = train_df["label"].replace(trans)
y_test = train_df["label"].replace(trans)
y_test

0       0
1       0
2       0
3       0
4       0
       ..
2632    9
2633    9
2634    9
2635    9
2636    9
Name: label, Length: 2637, dtype: int64

In [ ]:
import jieba
from urllib.request import urlretrieve
url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
urlretrieve(url, "dict.txt.big")
jieba.set_dictionary("dict.txt.big")
def newscut(p):
  return " ".join(jieba.cut(p))
x_train = train_df["label"].apply(newscut)
x_test = train_df["label"].apply(newscut)
x_test

Building prefix dict from /content/dict.txt.big ...
Dumping model to file cache /tmp/jieba.u501edca284da514cb68b53a20324f4e3.cache
Loading model cost 1.689 seconds.
Prefix dict has been built successfully.


0       軍事
1       軍事
2       軍事
3       軍事
4       軍事
        ..
2632    政治
2633    政治
2634    政治
2635    政治
2636    政治
Name: label, Length: 2637, dtype: object

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train_count = vec.fit_transform(x_train)
x_test_count = vec.fit_transform(x_test)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train_count, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test_count)
accuracy_score(pre, y_test)

1.0

In [ ]:
test = input("輸入一則新聞:")
count = vec.transform([newscut(test)])
answer = clf.predict(count)[0]
proba = clf.predict_proba(count)[0]
for name, p in zip(u, proba):
    print(name, "的機率", p)
print("答案:", trans_r[answer])